In [1]:
#m$6+".(RC6fC6H'
import csv
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
import pandas as pd


import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torchvision.transforms import functional as F

In [2]:
from facenet_pytorch import MTCNN, InceptionResnetV1

In [3]:
import wandb
wandb.init(project="my-test-project", entity="janghanju")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: janghanju (use `wandb login --relogin` to force relogin)


In [4]:
class IdModule(nn.Module):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return x

resnet = InceptionResnetV1(pretrained='vggface2').eval()
resnet.last_linear = nn.Linear(resnet.last_linear.in_features, 1)
resnet.last_bn = IdModule()
resnet.logits = IdModule()

In [10]:
help(InceptionResnetV1)

Help on class InceptionResnetV1 in module facenet_pytorch.models.inception_resnet_v1:

class InceptionResnetV1(torch.nn.modules.module.Module)
 |  InceptionResnetV1(pretrained=None, classify=False, num_classes=None, dropout_prob=0.6, device=None)
 |  
 |  Inception Resnet V1 model with optional loading of pretrained weights.
 |  
 |  Model parameters can be loaded based on pretraining on the VGGFace2 or CASIA-Webface
 |  datasets. Pretrained state_dicts are automatically downloaded on model instantiation if
 |  requested and cached in the torch cache. Subsequent instantiations use the cache rather than
 |  redownloading.
 |  
 |  Keyword Arguments:
 |      pretrained {str} -- Optional pretraining dataset. Either 'vggface2' or 'casia-webface'.
 |          (default: {None})
 |      classify {bool} -- Whether the model should output classification probabilities or feature
 |          embeddings. (default: {False})
 |      num_classes {int} -- Number of output classes. If 'pretrained' is s

In [5]:
transform =  transforms.Compose([                               
        transforms.Resize([256, 256]),                       
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(torch.float),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
test = Image.open('./train/18/989.jpg')
test = transform(test).unsqueeze(0)

/home/eis/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:127: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  img = torch.as_tensor(np.asarray(pic))


In [6]:
GPU_NUM = 0
IS_CUDA = torch.cuda.is_available()
#DEVICE  = 'cpu'
DEVICE = torch.device('cuda:' + str(GPU_NUM) if IS_CUDA else 'cpu')
DATASET_DIR = './train_v3'
BATCH_SIZE  = 64

LEARNING_RATE  = 5e-5
#L2WEIGHT_DECAY = 0.001
EPOCHS = 50

SaveModelName = "Woman_age_test"

wandb.config = {
  "learning_rate": 1e-5,
  "epochs": 50,
  "batch_size": 128
}

In [7]:
class DataManager():
    def __init__(self, dataset_dir, batch_size):
        self.batch_size  = batch_size
        self.dataset_dir = dataset_dir
    def Load_Dataset(self):
        transform =  transforms.Compose([                               
        transforms.Resize([224, 224]),                       
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(torch.float),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        dataset = ImageFolder(self.dataset_dir, transform=transform)
        print(f'Total data length : {len(dataset)}')
        test_size = len(dataset)//5
        train_size = len(dataset) - test_size

        #train_data, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])
        train_data, test_data = random_split(dataset, [train_size, test_size])

        print(f'train data size : {len(train_data)} , test data size :{len(test_data)}')
        return train_data, test_data
    
    def Load_DataLoader(self, train, test):
        return DataLoader(train, num_workers=16, batch_size=self.batch_size, shuffle=True, pin_memory=True, drop_last=False), \
                DataLoader(test, num_workers=16, batch_size=self.batch_size, shuffle=True, pin_memory=True, drop_last=False)


In [8]:
from resnet import resnet18, resnet34, resnet50


N_CLASSES = 53#18 ~ 70
EPOCHS = 80
def train_model(checkpoint_dir):
    if not os.path.exists(checkpoint_dir):
        os.mkdir(checkpoint_dir)
    
    DM = DataManager(DATASET_DIR, BATCH_SIZE)
    TRAIN_DATA, TEST_DATA = DM.Load_Dataset()
    TRAIN_LOADER, TEST_LOADER = DM.Load_DataLoader(TRAIN_DATA, TEST_DATA)
    
    MODEL = InceptionResnetV1(pretrained='vggface2',
                             classify=True,)
    MODEL.last_linear = nn.Linear(MODEL.last_linear.in_features, 1)
    MODEL.last_bn = IdModule()
    MODEL.logits = IdModule()

    #MODEL = nn.DataParallel(MODEL, output_device=0)
    optimizer = optim.Adam(MODEL.parameters(), lr = LEARNING_RATE)
    #criterion = nn.CrossEntropyLoss().to(DEVICE)
    criterion = nn.MSELoss().to(DEVICE)
    
    MODEL.to(DEVICE)
    total_loss = []
    total_acc = []
  
    for epoch in range(1, EPOCHS):
        MODEL.train()
        #print(f'epoch {epoch} starting')
        with tqdm(TRAIN_LOADER, unit='batch') as train_epoch:
            for i, (inputs, targets) in enumerate(train_epoch):
                #print(inputs.shape)
                inputs = inputs.to(DEVICE)
                targets = targets.to(DEVICE).float().unsqueeze(1)
                outputs = MODEL(inputs)
                #print(targets[0], outputs[0])
                loss = criterion(outputs, targets)#.to(torch.float32)#.to(torch.FloatTensor)#type(torch.FloatTensor)
                #print(type(loss))
                wandb.log({"loss": loss})
                
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                train_epoch.set_description(f'Training epoch {epoch}->')
        MODEL.eval()
        with torch.no_grad():
            with tqdm(TEST_LOADER, unit='batch') as test_epoch:
                for j, (val_inputs, val_targets) in enumerate(test_epoch):
                    #print(val_inputs.shape)
                    val_inputs = val_inputs.to(DEVICE)
                    
                    val_targets = val_targets.to(DEVICE).unsqueeze(1)
                    val_outputs = MODEL(val_inputs)
                    print(val_targets[0], val_outputs[0])
                    val_loss = criterion(val_outputs, val_targets)
                    val_targets_np =  val_targets.to('cpu').detach().numpy()
                    val_outputs_np = val_outputs.to('cpu').detach().numpy()
                    val_loss_mae = abs(val_targets_np - val_outputs_np)
                    wandb.log({"val_loss": val_loss})
                    wandb.watch(MODEL)
                    #val_outputs_np = np.argmax(val_outputs_np, axis =1).reshape(-1,1)
                    #val_acc  = metrics.accuracy_score(val_targets_np, val_outputs_np)
                    total_loss.append(val_loss.cpu().detach().numpy())
                    total_acc.append(np.mean(val_loss_mae))
                    test_epoch.set_description(f'Evaluating...->')
        total_loss_mean = np.mean(total_loss)
        total_acc_mean = np.mean(total_acc)
        print(f'loss {total_loss_mean:.3f} | Acc {total_acc_mean:.3f}\n') 
        
        if epoch % 2 == 0:
            print(f'result of epoch :{epoch} saving')
            torch.save({
            'epoch' : epoch,
            'model_state_dict': MODEL.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss_mean,
            'accuracy': total_acc_mean
            }, f"{checkpoint_dir}/last_checkpoint_part_v2_{epoch}.pth")

        
        
def evaluate_model(checkpoint_dir):
    DM = DataManager(DATASET_DIR, BATCH_SIZE)
    TRAIN_DATA, TEST_DATA = DM.Load_Dataset()
    TRAIN_LOADER, TEST_LOADER = DM.Load_DataLoader(TRAIN_DATA, TEST_DATA)
    
    criterion = nn.CrossEntropyLoss().to(DEVICE)
    #criterion = nn.CrossEntropyLoss().to(DEVICE)
    MODEL = InceptionResnetV1(pretrained='vggface2',
                             classify=True,)
    MODEL.last_linear = nn.Linear(MODEL.last_linear.in_features, 1)
    MODEL.last_bn = IdModule()
    MODEL.logits = IdModule()
    
    #MODEL = nn.DataParallel(MODEL, output_device=0)
    
    optimizer = optim.Adam(MODEL.parameters(), lr = LEARNING_RATE)
    #criterion = nn.CrossEntropyLoss().to(DEVICE)
    criterion = nn.MSELoss().to(DEVICE)
    
    LOAD_MODEL = True
    if LOAD_MODEL:
        checkpoint = torch.load(f"{checkpoint_dir}/last_checkpoint_2.pth")
        MODEL.load_state_dict(checkpoint["model_state_dict"])
    MODEL.to(DEVICE)
    MODEL.eval()
    result_pred, result_annotations = [], []
    LOSS_TRACE,  LOSS_TRACE_FOR_TEST = [], []

    with torch.no_grad():
        with tqdm(TEST_LOADER, unit='batch') as test_epoch:
            for val_inputs, val_targets in test_epoch:
                #print(val_inputs.shape, val_targets.shape)#, len(groups))
                val_inputs = val_inputs.to(DEVICE)
                val_targets = val_targets.to(DEVICE)
                val_outputs = MODEL(val_inputs)

                val_loss = criterion(val_outputs, val_targets)
                val_loss_np = val_loss.cpu().detach().numpy()
                LOSS_TRACE_FOR_TEST.append(val_loss_np)

                y_pred_test_np  = val_outputs.to('cpu').detach().numpy()
                y_pred_test_np  = np.argmax(y_pred_test_np, axis=1).reshape(-1, 1)
                y_test_np       = val_targets.to('cpu').detach().numpy().reshape(-1, 1)

                
                val_loss_mae = abs(val_targets_np - val_outputs_np)
                
                result_pred.extend(list(y_pred_test_np))
                result_annotations.extend(list(y_test_np))

                test_epoch.set_description(f'Evaluating...->')

    result_pred_np = np.array(result_pred).reshape(-1, 1)
    result_anno_np = np.array(result_annotations).reshape(-1, 1)
    print('--------------------------------------------------------------')
    ACC_TEST = metrics.accuracy_score(result_anno_np, result_pred_np)
    print('Accuracy: ', ACC_TEST)
    conf_mat = metrics.confusion_matrix(result_anno_np, result_pred_np)
    print(metrics.classification_report(result_anno_np, result_pred_np))
    print('Confustion Matrix: ')
    print(conf_mat)

    plt.rcParams["figure.figsize"] = (N_CLASSES, N_CLASSES)
    sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
    plt.show()
    conf_mat_sum = np.sum(conf_mat, axis=1)
    conf_mat_sum = np.reshape(conf_mat_sum, (N_CLASSES, 1))
    sns.heatmap(conf_mat/conf_mat_sum, annot=True, fmt='.2%', cmap='Blues')
    plt.show()
    print('--------------------------------------------------------------\n\n')      
        
        


In [9]:
# Tests
if __name__ == "__main__":
    checkpoint_dir = f'test_run'
    print("test_train")
    train_model(checkpoint_dir)
    #evaluate_model(checkpoint_dir)

test_train
Total data length : 6790
train data size : 5432 , test data size :1358


  0%|          | 0/85 [00:00<?, ?batch/s]

  0%|          | 0/22 [00:00<?, ?batch/s]

tensor([18], device='cuda:0') tensor([16.7460], device='cuda:0')
tensor([6], device='cuda:0') tensor([10.5125], device='cuda:0')
tensor([14], device='cuda:0') tensor([8.4625], device='cuda:0')
tensor([25], device='cuda:0') tensor([16.6277], device='cuda:0')
tensor([24], device='cuda:0') tensor([17.8209], device='cuda:0')
tensor([20], device='cuda:0') tensor([7.9929], device='cuda:0')
tensor([1], device='cuda:0') tensor([5.8893], device='cuda:0')
tensor([19], device='cuda:0') tensor([15.7287], device='cuda:0')
tensor([12], device='cuda:0') tensor([8.4890], device='cuda:0')
tensor([7], device='cuda:0') tensor([10.9208], device='cuda:0')
tensor([10], device='cuda:0') tensor([8.6047], device='cuda:0')
tensor([11], device='cuda:0') tensor([9.1711], device='cuda:0')
tensor([25], device='cuda:0') tensor([19.2308], device='cuda:0')
tensor([19], device='cuda:0') tensor([16.1004], device='cuda:0')
tensor([3], device='cuda:0') tensor([6.7709], device='cuda:0')
tensor([18], device='cuda:0') tensor

  0%|          | 0/85 [00:00<?, ?batch/s]

...

  0%|          | 0/85 [00:00<?, ?batch/s]

  0%|          | 0/22 [00:00<?, ?batch/s]

tensor([20], device='cuda:0') tensor([17.4806], device='cuda:0')
tensor([6], device='cuda:0') tensor([4.5384], device='cuda:0')
tensor([3], device='cuda:0') tensor([11.3591], device='cuda:0')
tensor([0], device='cuda:0') tensor([0.2391], device='cuda:0')
tensor([23], device='cuda:0') tensor([7.6390], device='cuda:0')
tensor([2], device='cuda:0') tensor([0.8326], device='cuda:0')
tensor([25], device='cuda:0') tensor([18.0740], device='cuda:0')
tensor([16], device='cuda:0') tensor([6.1394], device='cuda:0')
tensor([9], device='cuda:0') tensor([8.7102], device='cuda:0')
tensor([11], device='cuda:0') tensor([16.7906], device='cuda:0')
tensor([19], device='cuda:0') tensor([21.8074], device='cuda:0')
tensor([22], device='cuda:0') tensor([14.5473], device='cuda:0')
tensor([7], device='cuda:0') tensor([7.2987], device='cuda:0')
tensor([17], device='cuda:0') tensor([17.2604], device='cuda:0')
tensor([38], device='cuda:0') tensor([37.2819], device='cuda:0')
tensor([43], device='cuda:0') tensor([

  0%|          | 0/85 [00:00<?, ?batch/s]

KeyboardInterrupt: 